In [67]:
import torch
from torchvision import datasets, transforms, models
from torchvision.models import ResNet18_Weights, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import time
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [68]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


In [69]:
data_dir = '../data/'  # Substitua pelo caminho do seu diretório local

In [70]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# train_dir = os.path.join(data_dir, 'train')
# test_dir = os.path.join(data_dir, 'test')

train_dir = '../data/train/'
test_dir = '../data/test/'

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Modelo ResNet18

In [71]:
# Carregar modelo pré-treinado ResNet-18 e modificar a última camada
resnet = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, len(train_dataset.classes))  # Ajustar para o número de classes do dataset

# Mover o modelo para o dispositivo (GPU)
resnet = resnet.to(device)

In [72]:
print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Função de perda e otimizador

In [75]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# Treinamento dos Modelos

In [76]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, checkpoint_path='../model/checkpoints/checkpoint.pth'):
    start_epoch = 0

    # Carregar checkpoint se disponível
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Checkpoint loaded. Starting from epoch {start_epoch}")

    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        start_time = time.time()
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Mover dados para a GPU
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        epoch_time = time.time() - start_time

        print(f'Epoch {epoch + 1}/{num_epochs} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Time: {epoch_time:.2f}s')
        
        # Salvar checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, checkpoint_path)

    return model

In [77]:
resnet_trained = train_model(resnet, train_loader, criterion, optimizer, num_epochs=10, checkpoint_path='../model/checkpoints/checkpoint_resnet18.pth')

Epoch 1/10 Loss: 2.5964 Acc: 0.3859 Time: 179.78s
Epoch 2/10 Loss: 1.6767 Acc: 0.5729 Time: 170.56s
Epoch 3/10 Loss: 1.4276 Acc: 0.6289 Time: 172.91s
Epoch 4/10 Loss: 1.2813 Acc: 0.6609 Time: 182.08s
Epoch 5/10 Loss: 1.1745 Acc: 0.6889 Time: 176.56s
Epoch 6/10 Loss: 1.0933 Acc: 0.7085 Time: 180.98s
Epoch 7/10 Loss: 1.0171 Acc: 0.7265 Time: 171.26s
Epoch 8/10 Loss: 0.9574 Acc: 0.7422 Time: 173.52s
Epoch 9/10 Loss: 0.8983 Acc: 0.7548 Time: 170.02s
Epoch 10/10 Loss: 0.8489 Acc: 0.7661 Time: 175.15s


# Avaliação dos Modelos

In [82]:
def evaluate_and_generate_classification_report(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Mover dados para a GPU
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Gerar classification report
    report = classification_report(all_labels, all_preds, target_names=train_dataset.classes)
    print(f'Classification Report:\n{report}')

In [83]:
evaluate_and_generate_classification_report(resnet_trained, test_loader)

Classification Report:
                         precision    recall  f1-score   support

              apple_pie       0.58      0.55      0.56       200
         baby_back_ribs       0.69      0.68      0.68       200
                baklava       0.74      0.77      0.75       200
         beef_carpaccio       0.73      0.76      0.74       200
           beef_tartare       0.64      0.66      0.65       200
             beet_salad       0.58      0.60      0.59       200
               beignets       0.76      0.83      0.79       200
               bibimbap       0.82      0.83      0.82       200
          bread_pudding       0.49      0.59      0.54       200
      breakfast_burrito       0.72      0.47      0.57       200
             bruschetta       0.69      0.52      0.59       200
           caesar_salad       0.70      0.77      0.73       200
                cannoli       0.76      0.79      0.78       200
          caprese_salad       0.81      0.68      0.74       200
 

In [ ]:
torch.save(resnet_trained.state_dict(), '../model/resnet18_fine_tuned.pth')